# Imports

In [1]:
import torch
from torch.nn.utils import prune

from tqdm import tqdm

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from utils.mask_utils import calculate_mask
from utils.hessian_utils import calc_inverse_hessian
from utils.prehook_utils import put_input_hooks
import gc

# Constants

In [2]:
CALIBRATION_SIZE=128
TOKEN_LENGTH=1024
CALIBRATION_BATCH_SIZE=1

EPSILON = 1e-8
B = 128 
Bs = 128

#hyperparam test, remove later
EPOCH_COUNT = 10

#set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' 
MODEL_NAME = "opt-6.7b"

#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'facebook/{MODEL_NAME}')
#Load dataset
dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', streaming=True)

# Calibration

## Function

In [3]:
# run model on batches of calibration data, then concatenate inputs
def split_model_calibration(model):
    batch_sentences = []
    for i, data in tqdm(enumerate(iter(dataset['train'])), total=CALIBRATION_SIZE):
        if i < CALIBRATION_SIZE + 1:
            if len(batch_sentences) >= CALIBRATION_BATCH_SIZE:
                with torch.no_grad():
                    encoded_input = tokenizer(batch_sentences, return_tensors="pt",
                                              padding="max_length", max_length=TOKEN_LENGTH,
                                              truncation=True).to(device=DEVICE)
                    model(**encoded_input, labels=encoded_input.input_ids)
                    del encoded_input
                    torch.cuda.empty_cache()
                    batch_sentences = []
            batch_sentences.append(data['text'])
        else:
            break

# SparseGPT

## Prune models (SparseGPT)

In [ ]:
from utils.prune_utils import sparsegpt_prune

SPARSITIES = [0.2,0.3,0.5,0.7,0.9,1]#0.1, 0.2,0.3,0.5,0.7,0.9,1
    
for i, SPARSITY in enumerate(tqdm(SPARSITIES, total=len(SPARSITIES))):
    model = OPTForCausalLM.from_pretrained(f'facebook/{MODEL_NAME}', 
                                       output_attentions=True, 
                                       output_hidden_states=True).to(device=DEVICE)
    model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])
    !nvidia-smi
    # Calculate feature hessians only for the first iteration
    if i == 0:
        feature_hessians = {}
        #put_input_hooks(model=model, features=feature_hessians, storage_dir=storage_dir, offload_freq=10000, feature_storage_device='cpu')
        put_input_hooks(model=model, features=feature_hessians, feature_storage_device='cpu')
        split_model_calibration(model)
    
    # Prune using the sparseGPT method, saves as pruned_models/{model_name}-{SPARSENESS}.pt WITHOUT mask
    sparsegpt_prune(model, MODEL_NAME, feature_hessians, EPSILON, SPARSITY, B, Bs)
    !nvidia-smi
    del model
del feature_hessians

  0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Sat Feb 25 13:12:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   28C    P0    60W / 250W |  26355MiB / 40960MiB |     43%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   


  1%|          | 1/128 [00:00<00:30,  4.17it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

 51%|█████     | 65/128 [24:07<23:55, 22.79s/it]

## Finetune models (SparseGPT)

from finetune_utils import finetune_model

SPARSITIES = [0.2,0.3,0.5,0.7,0.9,1]#0.1, 0.2,0.3,0.5,0.7,0.9,1
    
for i, SPARSITY in enumerate(tqdm(SPARSITIES, total=len(SPARSITIES))):
    # Finetune, saves as pruned_models/{model_name}-{SPARSENESS}-finetuned.pt WITHOUT mask
    finetune_model(MODEL_NAME, tokenizer, SPARSITY, EPOCH_COUNT=EPOCH_COUNT)

## Test models (SparseGPT)

import wandb
wandb.login()

import numpy as np
from testing_module import test_model

TOKEN_LENGTH=1024
STRIDE = 512
wandb.init(project="ICLR", 
           name = f'{MODEL_NAME} Wikitext Test', 
           config={'token_length': TOKEN_LENGTH,
                 'model_name': MODEL_NAME,
                 'stride': STRIDE,
                 'fine_tuned': 'not finetuned'})
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'facebook/{MODEL_NAME}', 
                                          padding_side='left', 
                                          use_fast=False)
# Load dataset
test_set = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer("\n\n".join(test_set['text']), return_tensors='pt')

seq_len = encodings.input_ids.size(1)
SPARSITIES = [0.2, 0.3, 0.5, 0.7, 0.9, 1]#, 0.4, 0.6, 0.8, 1

for SPARSITY in SPARSITIES:
    test_model(model_name, encodings, token_length, seq_len, stride, wandb, SPARSITY, is_finetuned=False)
    
### NOW DO FINETUNED
wandb.init(project="ICLR", 
           name = f'{model_name} Wikitext Test', 
           config={'token_length': token_length,
                 'model_name': model_name,
                 'stride': stride,
                 'fine_tuned': 'finetuned'})
for SPARSITY in SPARSITIES:
    test_model(model_name, encodings, token_length, seq_len, stride, wandb, SPARSITY, is_finetuned=True)

# Cerebras